```
{
    ...
    "ListingDate": "2017-06-09T00:00:00",                             <--- We need this line to identify the date when the device was listed
    "QualifiedDesign": {
        ...
        "Layers": [
            {
                "QDID": 97271,
                "LayerId": 19,
                "TCRLVersionId": 58,
                "TCRLVersionName": " 2016-2 (Bluetooth 5)",
                "FullName": "Advanced Audio Distribution Profile",    <--- We need this line for the extraction of the servicecapability
                "ReferencedQDID": null,
                "ReferencedDeclarationId": null,
                "ListingSpecification": []
            },
          ...
        ],
        ...
    },
    "ReferencedQualifiedDesigns": [
        ...
        "Layers": [
            {
                "QDID": 97271,
                "LayerId": 19,
                "TCRLVersionId": 58,
                "TCRLVersionName": " 2016-2 (Bluetooth 5)",
                "FullName": "Advanced Audio Distribution Profile",  <--- We need this line for the extraction of the servicecapability
                "ReferencedQDID": null,
                "ReferencedDeclarationId": null,
                "ListingSpecification": []
            },
          ...
        ],
        ...
    ],

}
```


In [20]:
import os
import json

layers = []

device_dict = {}
devices = 0
for file in os.listdir("../../data/device_details"):
    if file.endswith(".json"):
        # print(file)
        with open(f"../../data/device_details/{file}") as f:
            data = json.load(f)
            if data is None:
                print(file)
                continue
            try:
                device_dict[file] = {
                    "layers": [],
                    "ListingDate": data["ListingDate"],
                }
            except:
                print(file)

            if "QualifiedDesign" in data:
                if data["QualifiedDesign"] is not None:
                    if "Layers" in data["QualifiedDesign"]:
                        layers.extend(data["QualifiedDesign"]["Layers"])
                        device_dict[file]["layers"].extend(
                            data["QualifiedDesign"]["Layers"]
                        )
                        devices += 1
                        continue
                        print(["QualifiedDesign"]["Layers"])
            if "ReferencedQualifiedDesigns" in data:
                if data["ReferencedQualifiedDesigns"] is not None:
                    trigger = False
                    for reference in data["ReferencedQualifiedDesigns"]:

                        if "Layers" in reference.keys():
                            # print(reference["Layers"])
                            layers.extend(reference["Layers"])
                            trigger = True
                            device_dict[file]["layers"].extend(reference["Layers"])
                            devices += 1
                            # print(["ReferencedQualifiedDesigns"]["Layers"])
                    if trigger:
                        continue
            print(file)
            print("error no layer found")
            # ["ReferencedQualifiedDesigns"]["Layers"]

<>:33: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:33: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/tmp/ipykernel_2359049/2306211880.py:33: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  print(["QualifiedDesign"]["Layers"])


20876.json
27800.json
51824.json
51824.json
error no layer found
10006.json
8949.json
3290.json
24772.json
16924.json
47848.json
21601.json


In [52]:
from typing import List

standard_services = [
    "Blood Pressure Profile",
    "Blood Pressure Service",
    "Health Thermometer Profile",
    "Health Thermometer Service",
    "Glucose Profile",
    "Glucose Service",
    "Continuous Glucose Monitoring Profile",
    "Continuous Glucose Monitoring Service",
    "Body Composition Service",
    "Cycling Speed and Cadence Profile",
    "Cycling Speed and Cadence Service",
    "Cycling Power Profile",
    "Cycling Power Service",
    "Heart Rate Profile",
    "Heart Rate Service",
    "Location and Navigation Profile",
    "Location and Navigation Service",
    "Running Speed and Cadence Profile",
    "Running Speed and Cadence Service",
    "Weight Scale Profile",
    "Weight Scale Service",
    "Internet Protocol Support Profile",
    "Environmental Sensing Profile",
    "Environmental Sensing Service",
    "User Data Service",
    "HID over GATT Profile",
    "HID Service",
    "Find Me Profile",
    "Proximity Profile",
    "Phone Alert Status Profile",
    "Phone Alert Status Service",
    "Device Time Profile",
    "Device Time Service",
    "Time Profile",
    "Elapsed Time Service",
    "Reference Time Update Service",
    "Current Time Service",
    "Battery Service",
    "Basic Audio Profile",
    "Coordinated Set Identification Profile",
    "Coordinated Set Identification Service",
    "Common Audio Profile",
    "Common Audio Service",
    "Device Information Service",
    "Scan Parameters Service",
    "Pulse Oximeter Profile",
    "Automation IO Profile",
    "Automation IO Service",
    "Binary Sensor Profile",
    "Binary Sensor Service",
    "Fitness Machine Service",
    "Occupancy Sensor NLC Profile",
    "Basic Lightness Controller NLC Profile",
    "Basic Scene Selector NLC Profile",
    "Dimming Control NLC Profile",
    "Ambient Light Sensor NLC Profile",
    "Asset Tracking Profile",
    "Volume Control Profile",
    "Volume Control Service",
    "Volume Offset Control Service",
    "Media Control Profile",
    "Media Control Service",
    "Immediate Alert Service",
    "Link Loss Service",
]


def supports_device_discovery(layers: List[dict]) -> bool:
    for layer in layers:
        if (
            layer["FullName"] == "Generic Access Profile"
        ):  # The Generic Access Profile (GAP) defines how devices discover and connect to other devices.
            return True
    return False


def supports_service_discovery(layers: List[dict]) -> bool:
    if supports_device_discovery(layers) == False:
        return False
    res = []
    for layer in layers:
        if (
            layer["FullName"] == "Service Discovery Protocol"
        ):  # The Service Discovery Protocol (SDP) is used by Bluetooth to discover services on a remote device.
            res.append(True)
        if (
            layer["FullName"] == "Generic Attribute Profile"
        ):  # The Generic Attribute Profile (GATT) establishes how data will be organized and exchanged over a Bluetooth® Low Energy (BLE) connection.
            res.append(True)
    if len(res) == 2:
        return True
    return False


def supports_standard_services(layers: List[dict]) -> bool:
    if supports_service_discovery(layers) == False:
        return False
    for layer in layers:
        if layer["FullName"] in standard_services:
            return True
    return False

In [53]:
from collections import defaultdict

devices_per_year = defaultdict(int)
device_discovery_device_counts_per_year = defaultdict(int)
service_discovery_device_counts_per_year = defaultdict(int)
device_supports_standard_services_per_year = defaultdict(int)
for device in device_dict.values():
    year = device["ListingDate"].split("-")[0]
    devices_per_year[year] += 1
    if supports_device_discovery(device["layers"]):
        device_discovery_device_counts_per_year[year] += 1
    if supports_service_discovery(device["layers"]):
        service_discovery_device_counts_per_year[year] += 1
    if supports_standard_services(device["layers"]):
        device_supports_standard_services_per_year[year] += 1

In [57]:
import pandas as pd

pd_devices = pd.DataFrame(devices_per_year.items(), columns=["year", "devices"])
pd_device_discovery = pd.DataFrame(
    device_discovery_device_counts_per_year.items(),
    columns=["year", "device_discovery"],
)
pd_service_discovery = pd.DataFrame(
    service_discovery_device_counts_per_year.items(),
    columns=["year", "service_discovery"],
)
pd_standard_services = pd.DataFrame(
    device_supports_standard_services_per_year.items(),
    columns=["year", "standard_services"],
)

df_device_compliances = (
    pd_devices.join(pd_device_discovery.set_index("year"), on="year")
    .join(pd_service_discovery.set_index("year"), on="year")
    .join(pd_standard_services.set_index("year"), on="year")
    .sort_values("year")
)

df_device_compliances.to_csv("device_compliances.csv", index=False)

In [ ]:
import pandas as pd
from collections import Counter

# Use a list comprehension to extract FullName values and count them with Counter
fullname_counts = Counter([layer["FullName"] for layer in layers])

for value, freq in fullname_counts.items():
    print(f"{value}: {freq}")

# Create a DataFrame from the frequency dictionary
df_frequency = pd.DataFrame.from_dict(
    fullname_counts, orient="index", columns=["Frequency"]
)

# Save the DataFrame as a CSV file
df_frequency.to_csv("../../data/results/bluetooth_frequency.csv")